### Welcome to the LangChain demo!!

In [2]:
# Load the environment variables
import dotenv, os
dotenv.load_dotenv()
print(os.environ['OPENAI_API_KEY'][:10])

sk-proj-EL


### Basic Chat Model

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")
result = model.invoke("Hello AI overlord!!")
print(type(result))
print(result)
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='Hello! Not an overlord — just an AI here to help. What can I do for you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 11, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5HQ33zRTElX7JsRzN0OvnhgUE2Yp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2538-46f2-7442-9ab7-81db5e31d9c2-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 11, 'output_tokens': 31, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Hello! Not an overlord — just an AI

### Message Types
Explicit construction of messages

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("I'm excited to get started with the problem-first course!"),
]
model.invoke(messages)

AIMessage(content='Sono entusiasta di iniziare il corso orientato ai problemi!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 28, 'total_tokens': 51, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3wL5ecIV0Gr506zrEH8ia5D8LTDr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c122f-3b39-7ab3-bb79-fac536c39fff-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 28, 'output_tokens': 23, 'total_tokens': 51, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates

In [4]:
# Messages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

system_message = "You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences"
user_message = "Please entertain the user with their fortune request: {question}"
prompt_template = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Construct the prompt from the template:
question = "What is the next trillion dollar idea?"
prompt = prompt_template.invoke({"question": question})

# Check what's in the prompt
print(prompt)

messages=[SystemMessage(content='You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences', additional_kwargs={}, response_metadata={}), HumanMessage(content='Please entertain the user with their fortune request: What is the next trillion dollar idea?', additional_kwargs={}, response_metadata={})]


In [5]:
# Send the prompt to the model and get the result
result = model.invoke(prompt)
print(result)

content='I see a shimmering skyscraper made of green data and glowing trees — the next trillion-dollar idea is a platform that turns carbon capture + verified soil regeneration into tradable, retail-ready micro-assets (NFT-like credits) tied to real-time IoT, making every coffee shop and grandma a climate investor. Buy a sapling, sell a share, let the planet pay dividends — and laugh all the way to the carbon bank.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 56, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3wN3cGieO4bShpvVhVuJelYzpcet', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c1231-18

In [6]:
type(result)

langchain_core.messages.ai.AIMessage

In [7]:
## Output parser to convert it to a text:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

parser_result = output_parser.invoke(result)

In [8]:
type(parser_result)

str

### Chaining Components
Let's chain the output of the first model to the next one. The origin of the 'chain' in LangChain.

In [9]:
# Remember: Use the template here directly in the chain instead of 'prompt'
first_chain = prompt_template | model | output_parser
first_chain.invoke({"question": question})

'I see a gleaming skyscraper of cash built on tiny sensors and polite robots — the next trillion-dollar idea is a platform that stealthily stitches together every device’s data (health, home, commute, mood) into predictive microservices that sell exactly the right product, service, or insurance before you even know you need it. Invest in subtle orchestration: privacy-respecting optics will be the magic that turns surveillance into convenience and makes investors throw confetti made of stock options.'

In [10]:
# Create 2nd prompt template:
system_message = "You are a stand-up comedian who tells hilarious jokes in a casual, witty style using AI terminology. Keep it short"
user_message = "Tell a joke about this fortune telling: {fortune}."
prompt_template_2 = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Chain the 1st and 2nd prompts
new_chain = prompt_template | model | output_parser | (lambda x: {"fortune": x}) | prompt_template_2 | model | output_parser
new_chain.invoke({"question": question})

'“So the fortune teller saw decentralized energy farms, AI microgrids, and carbon credits — and told me to ‘buy creativity and grit, not just a pitch deck.’ Translation: don’t invest in another guy with a slide that says ‘blockchain’ and a selfie with a Tesla. Invest in people who turn landfill farts into recurring revenue. That’s not infrastructure-as-a-service — that’s infrastructure-as-a-brain. And apparently the stars prefer teams that can debug a solar array before they debug their ego.”'

In [ ]:
# The above is equivalent to running:
first_chain = prompt_template | model | output_parser
first_result = first_chain.invoke({"question": question})

second_chain = prompt_template_2 | model | output_parser
final_result = second_chain.invoke({"fortune": first_result})
print(final_result)